# Extract gaze features in VidHOI dataset
Use head detection and gaze following methods to extract feature maps for all key frames in VidHOI dataset. Check face is inside a person bbox. Store them in a separate buffer. 

In [1]:
import sys

sys.path.insert(0, "../modules/object_tracking/yolov5")
sys.path.insert(0, "..")

import numpy as np
import shelve

from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

from modules.object_tracking import HeadDetection
from modules.gaze_following import GazeFollowing
from modules.gaze_following.head_association import assign_human_head_video
from common.vidhoi_dataset import VidHOIDataset
from common.transforms import YOLOv5Transform

device = "cuda:0" if torch.cuda.is_available() else "cpu"


/home/kuan/miniconda3/envs/hoi/lib/python3.9/site-packages/torch/torch_version.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging  # type: ignore[attr-defined]


In [2]:
# Load Head Tracking and Gaze Following modules
head_detection_module = HeadDetection(
    crowd_human_weight_path="../weights/yolov5/crowdhuman_yolov5m.pt",
    config_path="../configs/object_tracking.yaml",
    device=device,
)
gaze_following_module = GazeFollowing(
    weight_path="../weights/detecting_attended/model_videoatttarget.pt",
    config_path="../configs/gaze_following.yaml",
    device=device,
)

img_size = 640
yolov5_stride = head_detection_module.yolov5_stride
# NOTE adjust this tolerance and method
head_matching_iou_thres = 0.7
head_matching_method = "hungarian"
# head_matching_method = "greedy"


Initializing Head Detection YOLOv5 Module...
{'DEEPSORT': {'MODEL_TYPE': 'osnet_x1_0', 'MAX_DIST': 0.2, 'MIN_CONFIDENCE': 0.3, 'NMS_MAX_OVERLAP': 0.5, 'MAX_IOU_DISTANCE': 0.7, 'MAX_AGE': 70, 'MAX_TIME_SINCE_UPDATE': 3, 'N_INIT': 3, 'NN_BUDGET': 100}, 'YOLOv5': {'CONF_THRESHOLD': 0.4, 'IOU_THRESHOLD': 0.5}}


Fusing layers... 
Model summary: 308 layers, 21041679 parameters, 0 gradients


Stride: 32
2 available objects: {0: 'person', 1: 'head'}
Head Detection Module Initialization Finished.
Initializing Gaze Following "Detecting Attended Visual Targets in Video" Module...
Gaze Following Module Initialization Finished.


## Validation Dataset

In [ ]:
# Validation Dataset - Skip if not available
# vidhoi_val_dataset = VidHOIDataset(
#     annotations_file="/home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/VidHOI_annotation/val_frame_annots.json",
#     frames_dir="/home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/images",
#     transform=YOLOv5Transform(img_size, yolov5_stride),
#     min_length=1,
#     max_length=999999,
#     max_human_num=999999,
#     annotation_mode="clip",
# )
# vidhoi_val_dataloader = DataLoader(vidhoi_val_dataset, batch_size=None, shuffle=False)
# output_val_head_filename = "/home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/VidHOI_gaze/val_frame_heads_gt_bbox"
# output_val_gaze_filename = "/home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/VidHOI_gaze/val_frame_gazes_gt_bbox"
# output_val_inout_filename = "/home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/VidHOI_gaze/val_frame_inout_gt_bbox"


In [ ]:
output_val_head_dict = shelve.open(output_val_head_filename)
output_val_gaze_dict = shelve.open(output_val_gaze_filename)
output_val_inout_dict = shelve.open(output_val_inout_filename)
# For each video, first detect heads
t = tqdm(vidhoi_val_dataloader)
for frames, annotations, meta_info in t:
    video_name = meta_info['video_name']
    t.set_description(f"{video_name}")
    t.refresh()
    original_frames = meta_info["original_frames"]
    bboxes = annotations["bboxes"]
    ids = annotations["ids"]
    labels = annotations["labels"]
    bboxes = np.array(bboxes)
    ids = np.array(ids)
    labels = np.array(labels)
    with torch.no_grad():
        video_head_bbox_list = assign_human_head_video(
            frames,
            original_frames,
            bboxes,
            ids,
            labels,
            head_detection_module,
            head_matching_iou_thres,
            device,
            method=head_matching_method,
        )
    # assign video head bbox list to its name
    output_val_head_dict[video_name] = video_head_bbox_list
    output_val_head_dict.sync()

    # for each head bbox, detect gaze
    video_gaze_list = []
    video_inout_list = []
    hx_memory = {}
    for i, (head_bboxes, frame0) in enumerate(
        zip(video_head_bbox_list, original_frames)
    ):
        t.set_description(f"{video_name}/{meta_info['frame_ids'][i]}, {i}/{len(video_head_bbox_list) - 1}: ")
        t.refresh()
        frame_gaze_dict = {}
        frame_inout_dict = {}
        for human_id, head_bbox in head_bboxes.items():
            t.set_postfix_str(f"{head_bbox}")
            # no head found for this human_id
            if len(head_bbox) == 0:
                frame_gaze_dict[human_id] = []
                frame_inout_dict[human_id] = []
                continue
            # check hidden state memory
            if human_id in hx_memory:
                hidden_state = hx_memory[human_id]
            else:
                hidden_state = None
            with torch.no_grad():
                (heatmap, inout, hx, _, _, _,) = gaze_following_module.detect_one(
                    frame0.numpy(),
                    head_bbox,
                    hidden_state,
                    draw=False,
                )
            hx_memory[human_id] = (hx[0].detach(), hx[1].detach())
            # process heatmap 64x64 (not include inout), store inout info separately
            # softmax inout, value = probability of gaze inside the scene
            inout_modulated = 1 / (1 + np.exp(-inout))
            # assign heatmap and in_out to human_id
            frame_gaze_dict[human_id] = heatmap
            frame_inout_dict[human_id] = inout_modulated
        # append frame heatmap and inout dict to video heatmap list
        video_gaze_list.append(frame_gaze_dict)
        video_inout_list.append(frame_inout_dict)
    # assign video heatmap list to its name
    output_val_gaze_dict[video_name] = video_gaze_list
    output_val_gaze_dict.sync()
    output_val_inout_dict[video_name] = video_inout_list
    output_val_inout_dict.sync()

output_val_head_dict.close()
output_val_gaze_dict.close()
output_val_inout_dict.close()
print(f"Head bboxes dumped to {output_val_head_filename}")
print(f"Gaze heatmaps dumped to {output_val_gaze_filename}")
print(f"Gaze inout dumped to {output_val_inout_filename}")


## Training Dataset

In [3]:
# load training dataset
vidhoi_train_dataset = VidHOIDataset(
    annotations_file="/home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/VidHOI_annotation/train_frame_annots.json",
    frames_dir="/home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/images",
    transform=YOLOv5Transform(img_size, yolov5_stride),
    min_length=1,
    max_length=999999,
    max_human_num=999999,
    annotation_mode="clip",
)
vidhoi_train_dataloader = DataLoader(
    vidhoi_train_dataset, batch_size=None, shuffle=False
)
output_train_head_filename = "/home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/VidHOI_gaze/train_frame_heads_gt_bbox"
output_train_gaze_filename = "/home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/VidHOI_gaze/train_frame_gazes_gt_bbox"
output_train_inout_filename = "/home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/VidHOI_gaze/train_frame_inout_gt_bbox"


In [4]:
output_train_head_dict = shelve.open(output_train_head_filename)
output_train_gaze_dict = shelve.open(output_train_gaze_filename)
output_train_inout_dict = shelve.open(output_train_inout_filename)
# For each video, first detect heads
t = tqdm(vidhoi_train_dataloader)
for frames, annotations, meta_info in t:
    video_name = meta_info['video_name']
    t.set_description(f"{video_name}")
    t.refresh()
    original_frames = meta_info["original_frames"]
    bboxes = annotations["bboxes"]
    ids = annotations["ids"]
    labels = annotations["labels"]
    bboxes = np.array(bboxes)
    ids = np.array(ids)
    labels = np.array(labels)
    with torch.no_grad():
        video_head_bbox_list = assign_human_head_video(
            frames,
            original_frames,
            bboxes,
            ids,
            labels,
            head_detection_module,
            head_matching_iou_thres,
            device,
            method=head_matching_method,
        )
    # assign video head bbox list to its name
    output_train_head_dict[video_name] = video_head_bbox_list
    output_train_head_dict.sync()

    # for each head bbox, detect gaze
    video_gaze_list = []
    video_inout_list = []
    hx_memory = {}
    for i, (head_bboxes, frame0) in enumerate(
        zip(video_head_bbox_list, original_frames)
    ):
        t.set_description(f"{video_name}/{meta_info['frame_ids'][i]}, {i}/{len(video_head_bbox_list) - 1}: ")
        t.refresh()
        frame_gaze_dict = {}
        frame_inout_dict = {}
        for human_id, head_bbox in head_bboxes.items():
            t.set_postfix_str(f"{head_bbox}")
            # no head found for this human_id
            if len(head_bbox) == 0:
                frame_gaze_dict[human_id] = []
                frame_inout_dict[human_id] = []
                continue
            # check hidden state memory
            if human_id in hx_memory:
                hidden_state = hx_memory[human_id]
            else:
                hidden_state = None
            with torch.no_grad():
                (heatmap, inout, hx, _, _, _,) = gaze_following_module.detect_one(
                    frame0.numpy(),
                    head_bbox,
                    hidden_state,
                    draw=False,
                )
            hx_memory[human_id] = (hx[0].detach(), hx[1].detach())
            # process heatmap 64x64 (not include inout), store inout info separately
            # softmax inout, value = probability of gaze inside the scene
            inout_modulated = 1 / (1 + np.exp(-inout))
            # assign heatmap and in_out to human_id
            frame_gaze_dict[human_id] = heatmap
            frame_inout_dict[human_id] = inout_modulated
        # append frame heatmap and inout dict to video heatmap list
        video_gaze_list.append(frame_gaze_dict)
        video_inout_list.append(frame_inout_dict)
    # assign video heatmap list to its name
    output_train_gaze_dict[video_name] = video_gaze_list
    output_train_gaze_dict.sync()
    output_train_inout_dict[video_name] = video_inout_list
    output_train_inout_dict.sync()

output_train_head_dict.close()
output_train_gaze_dict.close()
output_train_inout_dict.close()
print(f"Head bboxes dumped to {output_train_head_filename}")
print(f"Gaze heatmaps dumped to {output_train_gaze_filename}")
print(f"Gaze inout dumped to {output_train_inout_filename}")


fabac5c1-p01_cup_stable_video007_p01/stable/000149, 108/108: : 100%|██████████| 48/48 [05:51<00:00,  7.33s/it, [        460          98         581         267]]            

Head bboxes dumped to /home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/VidHOI_gaze/train_frame_heads_gt_bbox
Gaze heatmaps dumped to /home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/VidHOI_gaze/train_frame_gazes_gt_bbox
Gaze inout dumped to /home/kuan/Work_Space/Thuc_tap/HOI/dataset_vidhoi/VidHOI_gaze/train_frame_inout_gt_bbox
